In [3]:
import pandas as pd
train = pd.read_csv('C:\Users\srikanthv\Downloads\data.csv', sep=',', header=0)

In [29]:
train['table-text'][0]

'<table id="selectedProduct">\n<tr>\n<th class="left"></th>\n<th class="title" colspan="2">Selected Product</th>\n<th>Price</th>\n<th>Make Selections</th>\n<th>qty</th>\n<th class="link"><a class="backtotop" href="#" id="btt">back to top</a></th>\n</tr>\n<tr valign="top">\n<td></td>\n<td class="img"> <!--requested size-%%%--><img alt="&amp;&amp;" border="0" css_style="" src="//cdn.gandermountain.com/product/746/swatches/746134_AQUA_1.jpg?interpolation=lanczos-none&amp;downsize=50px:*"/><!--using size-%%%-->\n</td>\n<td class="title">\n<h1>CORRCTCRAFT NTIQUE EXCEL BR DNT COVER</h1>\n<p class="item">Item # 746134</p>\n</td>\n<td class="price">\n<!--begin_item_pricing-->\n<!--special_parse-->\n<div class="price"> \xc2\xa0<span class="regstrk">$730.99</span></div>\n<div class="saleprice"> \xc2\xa0<span>$621.34</span></div>\n<!--end_item_pricing-->\n</td>\n<td class="options">\n<!--begin_lower_selects-->\n<!--special_parse-->\n<div class="attributes">\n<!--begin_select_style_box-->\n<!-- if

In [4]:
train[train['label'] == 'no'].count()
train.count()

label         91845
table-text    91845
url           91845
dtype: int64

In [5]:
from sklearn import cross_validation
train_split, cv_split = cross_validation.train_test_split(train, train_size=0.65)

In [6]:
train_split[train_split['label'] == 'yes'].count()

label         4196
table-text    4196
url           4196
dtype: int64

In [7]:
from sklearn.feature_extraction.text import  TfidfVectorizer

#str_replace = lambda str: str.replace('<', ' ').replace('>', ' ').replace('\n', ' ').replace('\r', ' ').replace('/', '')
str_replace = lambda str: str.replace('\n', ' ').replace('\r', ' ').replace('/', '')

train_split['table-text'] = train_split['table-text'].apply(str_replace)
cv_split['table-text'] = cv_split['table-text'].apply(str_replace)

#tfidf_transformer = TfidfVectorizer(ngram_range=(1,2),max_features=20000)
tfidf_transformer = TfidfVectorizer(ngram_range=(1,3), stop_words='english', max_df=0.5)

X_train_tfidf = tfidf_transformer.fit_transform(train_split['table-text'])
X_train_tfidf.shape

X_cv_tfidf = tfidf_transformer.transform(cv_split['table-text'])
print(X_cv_tfidf.shape)
print(X_train_tfidf.shape)


C:\Users\srikanthv\AppData\Local\Enthought\Canopy\User\lib\site-packages\ipykernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\srikanthv\AppData\Local\Enthought\Canopy\User\lib\site-packages\ipykernel\__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(32146, 1523944)
(59699, 1523944)


In [42]:
X_cv_tfidf[:,]

(32146, 100000)

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.decomposition import IncrementalPCA
from sklearn.metrics import accuracy_score
forest = RandomForestClassifier(n_estimators = 60 )

ch2 = SelectKBest(chi2, k=100000)
X_train_tfidf = ch2.fit_transform(X_train_tfidf, train_split['label'])
X_cv_tfidf = ch2.transform(X_cv_tfidf)

forest = forest.fit(X_train_tfidf, train_split['label'])
cv_predict = forest.predict(X_cv_tfidf)
accuracy_score(cv_split['label'], cv_predict)

0.99611149132084864

In [10]:
test = pd.read_csv('C:\Users\srikanthv\Desktop\\blindset_table_out.csv', sep=',', header=0)

In [11]:
test['table-text'] = test['table-text'].apply(str_replace)
X_test_tfidf = tfidf_transformer.transform(test['table-text'])
X_test_tfidf = ch2.transform(X_test_tfidf)
predict = forest.predict(X_test_tfidf)

In [13]:
import numpy as np
np.savetxt("new_random_forest_submission.csv", predict, fmt="%s")

In [8]:
predict

array(['no', 'no', 'no', ..., 'no', 'no', 'no'], dtype=object)

In [12]:
X_cv_tfidf

<32146x20000 sparse matrix of type '<type 'numpy.float64'>'
	with 5376836 stored elements in Compressed Sparse Row format>

In [14]:
cv_predict

array(['no', 'no', 'no', ..., 'no', 'no', 'no'], dtype=object)

In [16]:
np.savetxt("cross_validation.csv", cv_predict, fmt="%s")

In [17]:
cv_split[cv_split['label'] == 'no'].count()

label         29997
table-text    29997
url           29997
dtype: int64

In [21]:
(100-(cv_split[cv_split['label'] == 'yes'].count()/cv_split.count())*100)

label         93.314876
table-text    93.314876
url           93.314876
dtype: float64